# Capstone Project - Opening More Fitness Centers in Los Angeles, California

### IBM Applied Data Science Capstone by IBM/Coursera

# Table of Contents

+ Introduction: Business Problem
+ Data
+ Methodology
    + Data Gathering
    + Geocoding
    + Data Visualoization: Mapping Los Angeles
    + Exploring Neighborhoods With Foursquare API
    + Clustering Neighborhoods by Gym & Fitness Center Data
+ Results
+ Discussion
+ Conclusion

# Introduction: Business Problem

The goal of this project is to determine optimal locations for a new gym or fitness center within Los Angeles, California. Specifically, the end result is to compile a list of neighborhoods that can serve as target consumer areas for stakeholders interested in opening a gym or fitness center in Los Angeles, California. The neighborhoods of interest will be those with no gyms or fitness centers within 3000 meters from their center points. 

# Data

The following data resources will be used for this project:


+ The official list of neighborhoods in Los Angeles. 
    + Description: Names of each Los Angeles neighborhood are listed. The scope of this project does not extend to the greater Los Angeles area and is confined to the official neighborhood boundaries under Los Angeles. 
    + Data Source: geohub.lacity.org
+ Coordinates for each Los Angeles neighborhood.
    + Description: Latitude and longitude coordinates for the center of each Los Angeles neighborhood. These coordinates will be used to plot circle markers over a map of Los Angeles and to explore each of them.
    + Source: geopy.geocoder package
+ Venue Data in proximity of each Los Angeles neighborhood.
    + Description: Using Foursquare API a search query will be done against the coordinates for each Los Angeles neighborhood and venue data that is within a radius of 3000 meters of each neighborhood will be generated. This venue data will then be filtered to account for only data related to gyms and fitness centers.
    + Data source: Foursquare API

# Methodology

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!pip install geopy
!pip install geocoder
import geocoder
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 

from bs4 import BeautifulSoup # library to parse HTML and XML documents

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!pip install folium
import folium # map rendering library

print('Libraries imported.')

     |████████████████████████████████| 98 kB 7.3 MB/s  eta 0:00:01
     |████████████████████████████████| 93 kB 2.7 MB/s  eta 0:00:01
Libraries imported.


### Data Gathering

In [2]:
#link to LA neighborhood data uploaded in github repository
url = "https://github.com/c2barreto/Coursera_Captstone/blob/main/LA_Times_Neighborhood_Boundaries.csv" 

In [3]:
df_Neighbs = pd.read_html(url)

In [4]:
df_Neighbs[0]

,Unnamed: 0,OBJECTID,name
0,NaN,1,Adams-Normandie
1,NaN,2,Arleta
2,NaN,3,Arlington Heights
3,NaN,4,Atwater Village
4,NaN,5,Baldwin Hills/Crenshaw
5,NaN,6,Bel-Air
6,NaN,7,Beverly Crest
7,NaN,8,Beverly Grove
8,NaN,9,Beverlywood
9,NaN,10,Boyle Heights


In [5]:
#Pass table into a Pandas data frame.
df_LA = df_Neighbs[0]

In [6]:
#Drop unnecessary columns
df_LA.drop(["Unnamed: 0", "OBJECTID"], inplace=True, axis=1) 

In [7]:
#Rename column
df_LA.rename(columns = {'name':'Neighborhood'}, inplace = True)

In [8]:
df_LA.head()

,Neighborhood
0,Adams-Normandie
1,Arleta
2,Arlington Heights
3,Atwater Village
4,Baldwin Hills/Crenshaw


In [9]:
df_LA.shape

(114, 1)

### Geocoding

In [10]:
# defining a function to retrieve each LA neighborhood's coordinates
def get_latlng(LAneighborhood):
    # initialize a variable to None
    lat_lng_coords = None
    # loop until all neighborhood coordinates are retrieved using geocoder package
    while(lat_lng_coords is None):
        g = geocoder.arcgis('{}, Los Angeles, California'.format(LAneighborhood))
        lat_lng_coords = g.latlng
    return lat_lng_coords

In [11]:
# call the defined function to get the neighborhood coordinates and then store the values into a list
coordList = [ get_latlng(LAneighborhood) for LAneighborhood in df_LA["Neighborhood"].tolist() ]

In [12]:
coordList

[[34.07809005053511, -118.30119989733268],
 [34.24905000000007, -118.43348999999995],
 [34.03989000000007, -118.32541999999995],
 [34.11970000000008, -118.25886999999994],
 [34.01043000000004, -118.33665999999994],
 [34.08361000000008, -118.43482999999998],
 [34.09653268819107, -118.40329514585207],
 [34.09544836979449, -118.42608686997163],
 [34.05138000000005, -118.40132999999997],
 [34.04004000000003, -118.21049999999997],
 [34.06626000000006, -118.47032999999999],
 [33.960930000000076, -118.27881999999994],
 [34.20239000000004, -118.60155999999995],
 [34.17639681534724, -118.22274327896007],
 [33.997420000000034, -118.24777999999998],
 [34.05944000000005, -118.41280999999998],
 [34.257250000000056, -118.59100999999998],
 [34.241840000000025, -118.62292999999994],
 [33.99251000000004, -118.31284999999997],
 [34.03403000000003, -118.41055999999998],
 [34.062290000000075, -118.23836999999997],
 [34.09448000000003, -118.22677999999996],
 [33.99082000000004, -118.42061999999999],
 [34.0

In [13]:
# create temporary dataframe to populate the coordinates
df_temp = pd.DataFrame(coordList, columns=['Latitude', 'Longitude'])

In [14]:
# merge the temp dataframe with the df_LA
df_LA['Latitude'] = df_temp['Latitude']
df_LA['Longitude'] = df_temp['Longitude']

In [15]:
df_LA

,Neighborhood,Latitude,Longitude
0,Adams-Normandie,34.078090,-118.301200
1,Arleta,34.249050,-118.433490
2,Arlington Heights,34.039890,-118.325420
3,Atwater Village,34.119700,-118.258870
4,Baldwin Hills/Crenshaw,34.010430,-118.336660
5,Bel-Air,34.083610,-118.434830
6,Beverly Crest,34.096533,-118.403295
7,Beverly Grove,34.095448,-118.426087
8,Beverlywood,34.051380,-118.401330
9,Boyle Heights,34.040040,-118.210500


In [16]:
df_LA.to_csv("df_LA3.csv", index=False)

### Data Visualization: Mapping Los Angeles

In [17]:
# Get the coordinates of Los Angeles
address = 'Los Angeles, CA'

geolocator = Nominatim(user_agent="ca_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Los Angeles are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Los Angeles are 34.0536909, -118.2427666.


In [18]:
# create map of Toronto using latitude and longitude values
map_LA = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, neighborhood in zip(df_LA['Latitude'], df_LA['Longitude'], df_LA['Neighborhood']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_LA)  
    
map_LA

In [19]:
# save the map as HTML file
map_LA.save('map_LA3.html')

### Exploring Neighborhoods With Foursquare API

In [20]:
# defining foursquare credentials and version
CLIENT_ID = 'H0ZIZZAEGBFDEGUJYC0XZKX4Z2ROHFX53WN0V5SENBVMDBVU' # my Foursquare ID
CLIENT_SECRET = 'D1HOVQ0X0UQV1JHXSZ0TIC0HFODDSELZH2WTWLJ0U5GQ5E1A' # my Foursquare Secret
VERSION = '20201105' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: H0ZIZZAEGBFDEGUJYC0XZKX4Z2ROHFX53WN0V5SENBVMDBVU
CLIENT_SECRET:D1HOVQ0X0UQV1JHXSZ0TIC0HFODDSELZH2WTWLJ0U5GQ5E1A


In [21]:
#defining a function to get top venues of each neighborhood within a 500 meter radius
def getNearbyVenues(names, latitudes, longitudes, radius=2000):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
        
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [22]:
# code to run the above function on each neighborhood and create a new dataframe called losangeles_venues
losangeles_venues = getNearbyVenues(names=df_LA['Neighborhood'],
                                   latitudes=df_LA['Latitude'],
                                   longitudes=df_LA['Longitude']
                                  )

Adams-Normandie
Arleta
Arlington Heights
Atwater Village
Baldwin Hills/Crenshaw
Bel-Air
Beverly Crest
Beverly Grove
Beverlywood
Boyle Heights
Brentwood
Broadway-Manchester
Canoga Park
Carthay
Central-Alameda
Century City
Chatsworth
Chatsworth Reservoir
Chesterfield Square
Cheviot Hills
Chinatown
Cypress Park
Del Rey
Downtown
Eagle Rock
East Hollywood
Echo Park
El Sereno
Elysian Park
Elysian Valley
Encino
Exposition Park
Fairfax
Florence
Glassell Park
Gramercy Park
Granada Hills
Green Meadows
Griffith Park
Hancock Park
Hansen Dam
Harbor City
Harbor Gateway
Harvard Heights
Harvard Park
Highland Park
Historic South-Central
Hollywood Hills
Hollywood Hills West
Hollywood
Hyde Park
Jefferson Park
Koreatown
Lake View Terrace
Larchmont
Leimert Park
Lincoln Heights
Los Feliz
Manchester Square
Mar Vista
Mid-City
Mid-Wilshire
Mission Hills
Montecito Heights
Mount Washington
North Hills
North Hollywood
Northridge
Pacific Palisades
Pacoima
Palms
Panorama City
Pico-Robertson
Pico-Union
Playa del Rey

In [23]:
print(losangeles_venues.shape)
losangeles_venues.head()

(9586, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Adams-Normandie,34.07809,-118.3012,Noshi Sushi,34.076159,-118.305374,Sushi Restaurant
1,Adams-Normandie,34.07809,-118.3012,Jaraguá,34.076364,-118.306646,Cocktail Bar
2,Adams-Normandie,34.07809,-118.3012,Kim Sun Young Hair Beauty Salon (Kim Sun Young...,34.076453,-118.308921,Salon / Barbershop
3,Adams-Normandie,34.07809,-118.3012,Guatemalteca Bakery,34.076303,-118.297168,Restaurant
4,Adams-Normandie,34.07809,-118.3012,Cactus Mexican Food,34.076194,-118.304147,Mexican Restaurant


In [24]:
#Checking how many venues were returned for each neighborhood
losangeles_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Adams-Normandie,100,100,100,100,100,100
Arleta,56,56,56,56,56,56
Arlington Heights,100,100,100,100,100,100
Atwater Village,100,100,100,100,100,100
Baldwin Hills/Crenshaw,75,75,75,75,75,75
Bel-Air,62,62,62,62,62,62
Beverly Crest,41,41,41,41,41,41
Beverly Grove,19,19,19,19,19,19
Beverlywood,100,100,100,100,100,100


In [25]:
# checking how many unique categories can be curated from all the returned venues
print('There are {} uniques categories.'.format(len(losangeles_venues['Venue Category'].unique())))

There are 413 uniques categories.


In [27]:
# one hot encoding
losangeles_onehot = pd.get_dummies(losangeles_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
losangeles_onehot['Neighborhood'] = losangeles_venues['Neighborhood'] 

# move neighborhood column to the first column
# define a list of column names
cols = losangeles_onehot.columns.tolist()
cols

# move the column name to the beggining
cols.insert(0, cols.pop(cols.index('Neighborhood')))
cols

#then use .reindex() function to reorder
losangeles_onehot = losangeles_onehot.reindex(columns= cols)

#check result
losangeles_onehot.head()


,Neighborhood,ATM,Accessories Store,Adult Boutique,Airport,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Amphitheater,Andhra Restaurant,Antique Shop,Aquarium,Arcade,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Astrologer,Athletics & Sports,Auditorium,Australian Restaurant,Auto Dealership,Auto Garage,Automotive Shop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Big Box Store,Bike Rental / Bike Share,Bike Shop,Bike Trail,Bistro,Board Shop,Boat or Ferry,Bookstore,Botanical Garden,Boutique,Bowling Alley,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bridge,Bubble Tea Shop,Buffet,Building,Burger Joint,Burmese Restaurant,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Cable Car,Cafeteria,Café,Cajun / Creole Restaurant,Camera Store,Campground,Canal,Candy Store,Caribbean Restaurant,Cave,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Churrascaria,Circus,City Hall,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Academic Building,College Administrative Building,College Arts Building,College Bookstore,College Cafeteria,College Classroom,College Football Field,College Gym,College Quad,College Rec Center,College Residence Hall,College Stadium,College Theater,Comedy Club,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Cycle Studio,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Disc Golf,Discount Store,Distribution Center,Dive Bar,Dive Shop,Doctor's Office,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dry Cleaner,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,English Restaurant,Escape Room,Ethiopian Restaurant,Event Space,Fabric Shop,Factory,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Film Studio,Fish & Chips Shop,Fish Market,Fishing Store,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Service,Food Stand,Food Truck,Football Stadium,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General College & University,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Go Kart Track,Golf Course,Golf Driving Range,Gourmet Shop,Government Building,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Halal Restaurant,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,High School,Himalayan Restaurant,Historic Site,History Museum,Hobby Shop,Home Service,Hookah Bar,Hospital,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotel Pool,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Indonesian Restaurant,Insurance Office,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Kebab Restaurant,Kids Store,Kitchen Supply Store,Korean Restaurant,Kosher Restaurant,Lake,Latin American Restaurant,Laundromat,Laundry Service,Lebanese Restaurant,Library,Light Rail Station,Lingerie Store,Liquor Store,Locksmith,Lounge,Marijuana Dispensary,Market,Martial Arts School,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Mongolian Restaurant,Monument / Landmark,Moroccan Restaurant,Motel,Motorcycle Shop,Mountain,Movie Theater,Moving Target,Multiplex,Museum,Music Store,Music Venue,Nail Salon,New American Restaurant,Nightclub,Non-Profit,Noodle House,North Indian Restaurant,Observatory,Office,Optical Shop,Organic Grocery,Other Great Outdoors,Other Repair Shop,Outdoor Sculpture,Outdoors & Recreation,O

In [28]:
#viewing the new dataframe size
losangeles_onehot.shape

(9586, 413)

In [29]:
#Grouping the rows by neighborhood and by taking the mean of the frequency of occurrence of each category
losangeles_grouped = losangeles_onehot.groupby('Neighborhood').mean().reset_index()
losangeles_grouped

,Neighborhood,ATM,Accessories Store,Adult Boutique,Airport,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Amphitheater,Andhra Restaurant,Antique Shop,Aquarium,Arcade,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Astrologer,Athletics & Sports,Auditorium,Australian Restaurant,Auto Dealership,Auto Garage,Automotive Shop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Big Box Store,Bike Rental / Bike Share,Bike Shop,Bike Trail,Bistro,Board Shop,Boat or Ferry,Bookstore,Botanical Garden,Boutique,Bowling Alley,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bridge,Bubble Tea Shop,Buffet,Building,Burger Joint,Burmese Restaurant,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Cable Car,Cafeteria,Café,Cajun / Creole Restaurant,Camera Store,Campground,Canal,Candy Store,Caribbean Restaurant,Cave,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Churrascaria,Circus,City Hall,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Academic Building,College Administrative Building,College Arts Building,College Bookstore,College Cafeteria,College Classroom,College Football Field,College Gym,College Quad,College Rec Center,College Residence Hall,College Stadium,College Theater,Comedy Club,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Cycle Studio,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Disc Golf,Discount Store,Distribution Center,Dive Bar,Dive Shop,Doctor's Office,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dry Cleaner,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,English Restaurant,Escape Room,Ethiopian Restaurant,Event Space,Fabric Shop,Factory,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Film Studio,Fish & Chips Shop,Fish Market,Fishing Store,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Service,Food Stand,Food Truck,Football Stadium,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General College & University,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Go Kart Track,Golf Course,Golf Driving Range,Gourmet Shop,Government Building,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Halal Restaurant,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,High School,Himalayan Restaurant,Historic Site,History Museum,Hobby Shop,Home Service,Hookah Bar,Hospital,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotel Pool,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Indonesian Restaurant,Insurance Office,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Kebab Restaurant,Kids Store,Kitchen Supply Store,Korean Restaurant,Kosher Restaurant,Lake,Latin American Restaurant,Laundromat,Laundry Service,Lebanese Restaurant,Library,Light Rail Station,Lingerie Store,Liquor Store,Locksmith,Lounge,Marijuana Dispensary,Market,Martial Arts School,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Mongolian Restaurant,Monument / Landmark,Moroccan Restaurant,Motel,Motorcycle Shop,Mountain,Movie Theater,Moving Target,Multiplex,Museum,Music Store,Music Venue,Nail Salon,New American Restaurant,Nightclub,Non-Profit,Noodle House,North Indian Restaurant,Observatory,Office,Optical Shop,Organic Grocery,Other Great Outdoors,Other Repair Shop,Outdoor Sculpture,Outdoors & Recreation,O

### Clustering Neighborhoods by Gym & Fitness Center Data

In [30]:
#confirming new size
losangeles_grouped.shape

(114, 413)

In [31]:
# count of neighborhoods with at least 1 occurance of a gym or fitness center near them
len(losangeles_grouped[losangeles_grouped["Gym / Fitness Center"] > 0])

47

In [32]:
# new data frame for LA neighborhood data on Gyms and Fitness Centers
LA_gyms = losangeles_grouped[["Neighborhood","Gym / Fitness Center"]]

In [33]:
LA_gyms.head()

,Neighborhood,Gym / Fitness Center
0,Adams-Normandie,0.00
1,Arleta,0.00
2,Arlington Heights,0.00
3,Atwater Village,0.02
4,Baldwin Hills/Crenshaw,0.00


In [34]:
# set number of clusters
kclusters = 5

losangeles_grouped_clustering = LA_gyms.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(losangeles_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:100] 

array([1, 1, 1, 3, 1, 1, 3, 1, 1, 1, 2, 1, 1, 4, 1, 1, 0, 1, 3, 3, 1, 1,
       2, 0, 0, 1, 3, 1, 1, 1, 0, 1, 0, 1, 4, 1, 1, 1, 1, 0, 1, 0, 1, 1,
       0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1,
       0, 3, 1, 1, 1, 3, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 4, 3, 1, 2,
       1, 1, 1, 3, 1, 1, 1, 0, 0, 1, 2, 1], dtype=int32)

In [35]:
LA_merged = LA_gyms.copy()
# add clustering labels column 
LA_merged["Cluster Labels"] = kmeans.labels_

LA_merged.head() # check the last columns!

,Neighborhood,Gym / Fitness Center,Cluster Labels
0,Adams-Normandie,0.00,1
1,Arleta,0.00,1
2,Arlington Heights,0.00,1
3,Atwater Village,0.02,3
4,Baldwin Hills/Crenshaw,0.00,1


In [36]:
# merge df_LA with LA_merged to add latitude/longitude for each neighborhood
LA_merged = LA_merged.join(df_LA.set_index("Neighborhood"), on="Neighborhood")

print(LA_merged.shape)
LA_merged.head() # check the last columns!

(114, 5)


,Neighborhood,Gym / Fitness Center,Cluster Labels,Latitude,Longitude
0,Adams-Normandie,0.00,1,34.07809,-118.30120
1,Arleta,0.00,1,34.24905,-118.43349
2,Arlington Heights,0.00,1,34.03989,-118.32542
3,Atwater Village,0.02,3,34.11970,-118.25887
4,Baldwin Hills/Crenshaw,0.00,1,34.01043,-118.33666


In [37]:
#visualize the resulting clusters
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(LA_merged['Latitude'], LA_merged['Longitude'], LA_merged['Neighborhood'], LA_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster)-1],
        fill=True,
        fill_color=rainbow[int(cluster)-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [38]:
map_clusters.save('map_clusters.html')

# Results

### Cluster 0 (Red)

In [41]:
cluster0 = LA_merged.loc[LA_merged['Cluster Labels'] == 0]
cluster0

,Neighborhood,Gym / Fitness Center,Cluster Labels,Latitude,Longitude
16,Chatsworth,0.010101,0,34.257250,-118.591010
23,Downtown,0.010000,0,34.048340,-118.255640
24,Eagle Rock,0.010000,0,34.139270,-118.210870
30,Encino,0.010000,0,34.165380,-118.527110
32,Fairfax,0.010000,0,34.076100,-118.361220
39,Hancock Park,0.010000,0,34.068920,-118.332550
41,Harbor City,0.010753,0,33.797810,-118.302480
44,Harvard Park,0.010000,0,34.144400,-118.252970
45,Highland Park,0.010000,0,34.109820,-118.192320
47,Hollywood,0.010000,0,34.101560,-118.337080


In [42]:
cluster0.shape

(30, 5)

Cluster 0 groups together 30 neighborhoods with a low concentration of fitness centers in proximity. The venue data for these neighborhoods generated at least 1 occurrence of a fitness center within a 2000 meter radius from their center points.

### Cluster 1 (Purple)

In [43]:
cluster1 = LA_merged.loc[LA_merged['Cluster Labels'] == 1]
cluster1

,Neighborhood,Gym / Fitness Center,Cluster Labels,Latitude,Longitude
0,Adams-Normandie,0.0,1,34.078090,-118.301200
1,Arleta,0.0,1,34.249050,-118.433490
2,Arlington Heights,0.0,1,34.039890,-118.325420
4,Baldwin Hills/Crenshaw,0.0,1,34.010430,-118.336660
5,Bel-Air,0.0,1,34.083610,-118.434830
7,Beverly Grove,0.0,1,34.095448,-118.426087
8,Beverlywood,0.0,1,34.051380,-118.401330
9,Boyle Heights,0.0,1,34.040040,-118.210500
11,Broadway-Manchester,0.0,1,33.960930,-118.278820
12,Canoga Park,0.0,1,34.202390,-118.601560


In [44]:
cluster1.shape

(67, 5)

Cluster 1 groups together 67 neighborhoods with no existence of fitness centers in proximity. The venue data for these neighborhoods generated 0 occurrences of a fitness center within a 2000 meter radius from their center points.

### Cluster 2 (Blue)

In [45]:
cluster2 = LA_merged.loc[LA_merged['Cluster Labels'] == 2]
cluster2

,Neighborhood,Gym / Fitness Center,Cluster Labels,Latitude,Longitude
10,Brentwood,0.031915,2,34.06626,-118.47033
22,Del Rey,0.030000,2,33.99082,-118.42062
87,Studio City,0.030000,2,34.14453,-118.39561
98,Venice,0.030000,2,33.98754,-118.47215


In [46]:
cluster2.shape

(4, 5)

Cluster 2 groups together 4 neighborhoods with a mid-high concentration of fitness centers in proximity. The venue data for these neighborhoods generated 3 occurrences of a fitness center within a 2000 meter radius from their center points.

## Cluster 3 (Green)

In [47]:
cluster3 = LA_merged.loc[LA_merged['Cluster Labels'] == 3]
cluster3

,Neighborhood,Gym / Fitness Center,Cluster Labels,Latitude,Longitude
3,Atwater Village,0.020000,3,34.119700,-118.258870
6,Beverly Crest,0.024390,3,34.096533,-118.403295
18,Chesterfield Square,0.016129,3,33.992510,-118.312850
19,Cheviot Hills,0.020000,3,34.034030,-118.410560
26,Echo Park,0.020000,3,34.076090,-118.255810
67,North Hollywood,0.020000,3,34.169820,-118.378990
71,Palms,0.020000,3,34.019860,-118.420960
85,Silver Lake,0.020000,3,34.082730,-118.273070
91,Tarzana,0.020000,3,34.175290,-118.550100
113,Woodland Hills,0.020000,3,34.168890,-118.611480


In [48]:
cluster3.shape

(10, 5)

Cluster 3 groups together 10 neighborhoods with a low-mid concentration of fitness centers in proximity. The venue data for these neighborhoods generated at least 2 occurrences of a fitness center within a 2000 meter radius from their center points.

### Cluster 4 (Orange)

In [49]:
cluster4 = LA_merged.loc[LA_merged['Cluster Labels'] == 4]
cluster4

,Neighborhood,Gym / Fitness Center,Cluster Labels,Latitude,Longitude
13,Carthay,0.047619,4,34.176397,-118.222743
34,Glassell Park,0.038462,4,34.119220,-118.230440
84,Sherman Oaks,0.040000,4,34.151060,-118.444340


In [50]:
cluster4.shape

(3, 5)

Cluster 4 groups together 3 neighborhoods with a high concentration of fitness centers in proximity. The venue data for these neighborhoods generated at least 4 occurrences of a fitness center within a 2000 meter radius from their center points.

# Discussion

Based on the clustering data over the map of Los Angeles, the majority of neighborhoods under Los Angeles county lack access to nearby fitness centers. Very few neighborhoods harbor a mid to high concentration of local fitness centers present. In fact, clusters 2, 3 and 4, which contain a mid to high concentration of fitness centers, can be combined and they would only total 17 neighborhoods out of the 114. Unsurprisingly, clusters 2, 3, and 4 contain suburban neighborhoods with a higher income population. Judging also by their higher concentration of fitness centers, it is safe to recommend to avoid building new fitness centers in these areas since they already have competition established. Even more so, the intent is to give more local access to neighborhoods that lack fitness centers.

Continuing on, the second largest cluster is cluster 0, which list neighborhoods with at least one occurrence of a fitness center in proximity. Cluster 0 contained 30 neighborhoods out of the 114 in Los Angeles and geographically was concentrated towards Western Los Angeles and North West Los Angeles. Competition for memberships is probably not high in these neighborhoods so they can be considered as a secondary tier of neighborhoods for building more fitness centers around. 

Every neighborhood that did not generate an occurence of a fitness center within a 2000 meter radius from their center point was assigned to Cluster 1. Cluster 1 had a total of 67 out of 114 neighborhoods assigned to its cluster. Geographically, the neighborhoods under cluster 1 that are located in the most southern side of Los Angeles have the least availability to access a fitness center in proximity.This discrepancy could be due to the fact high concentration of low income residents reside in the southern side of Los Angeles which in the past might have deterred property developers from establishing fitness centers there. 

In Addition, there are also several other notable geographic concentrations from cluster 1 that display a lack of available neighborhoods such as East Los Angeles, Central Los Angeles, and the most northern regions of Los Angeles that hug San Fernando. These city regions from cluster 1 are also high population dense areas so there is definitely a high probability of demand for more convenient fitness centers in these areas. 



# Conclusion

Despite there being some limitations as far as how much venue data can be queried from Foursquare, enough venue data was gathered where an accurate mean frequency of occurrence for gyms and fitness centers can be obtained for each Los Angeles neighborhood. From these mean frequency of occurrences for gyms and fitness centers, five unique clusters were able to be made and each held notable geographic insights. In Particular the most stand out cluster that can be referred to as the ideal list of neighborhoods to target for building new fitness centers is cluster 1. Cluster 1 revealed that there are at least three big and populous regions in Los Angeles that have zero fitness centers established and they are Southern Los Angeles, East los Angeles, and Central Los Angeles. While these regions have historically housed low-income residents, this factor should not limit property developers from pursuing to build fitness centers in these regions. In fact, property developers should aim to build a market of low-cost membership fitness centers in these regions. Better yet, the city of Los Angeles can serve the health needs of its low-income residents better by building community fitness recreational centers in these neighborhoods as well. Overall, there is a market and need to build more fitness centers for the neighborhoods listed under cluster 1. 
